In [1]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

#pl.Config.set_tbl_rows(2000)

req = GetTanksTimeDataReq(source_key=['9883901', '98839FAC'])

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

ts,value,source_key,metric_nice_name,uom
datetime[μs],f64,str,str,str
2024-08-22 20:10:28.260583,291.456951,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-26 18:34:13.231329,298.883254,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-28 06:15:32.821494,262.238814,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-26 09:32:03.681955,303.781195,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-29 10:53:08.232747,351.712976,"""9883901""","""OilTank1Volume""","""bbl"""
…,…,…,…,…
2024-08-28 22:23:33.615018,232.012233,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in"""
2024-08-28 13:59:08.358302,198.544071,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in"""
2024-08-29 07:27:55.213780,272.102104,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in"""


In [2]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

ts,value,source_key,metric_nice_name,uom,is_esd,tank_type,tank_number,tank_metric
datetime[μs],f64,str,str,str,str,str,str,str
2024-08-22 20:10:28.260583,291.456951,"""9883901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2024-08-26 18:34:13.231329,298.883254,"""9883901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2024-08-28 06:15:32.821494,262.238814,"""9883901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2024-08-26 09:32:03.681955,303.781195,"""9883901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2024-08-29 10:53:08.232747,351.712976,"""9883901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
…,…,…,…,…,…,…,…,…
2024-08-28 22:23:33.615018,232.012233,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""ESD-""","""Oil""","""""","""InchesUntilAlarm"""
2024-08-28 13:59:08.358302,198.544071,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""ESD-""","""Oil""","""""","""InchesUntilAlarm"""
2024-08-29 07:27:55.213780,272.102104,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""ESD-""","""Oil""","""""","""InchesUntilAlarm"""


In [3]:
lf = lf.drop("is_esd", "tank_number")
lf.collect()

ts,value,source_key,metric_nice_name,uom,tank_type,tank_metric
datetime[μs],f64,str,str,str,str,str
2024-08-22 20:10:28.260583,291.456951,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-26 18:34:13.231329,298.883254,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-28 06:15:32.821494,262.238814,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-26 09:32:03.681955,303.781195,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-29 10:53:08.232747,351.712976,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
…,…,…,…,…,…,…
2024-08-28 22:23:33.615018,232.012233,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-28 13:59:08.358302,198.544071,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-29 07:27:55.213780,272.102104,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""


In [4]:
filtered_lf = lf.filter(~pl.col("metric_nice_name").str.ends_with("ID"))
filtered_lf.collect()

ts,value,source_key,metric_nice_name,uom,tank_type,tank_metric
datetime[μs],f64,str,str,str,str,str
2024-08-22 20:10:28.260583,291.456951,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-26 18:34:13.231329,298.883254,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-28 06:15:32.821494,262.238814,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-26 09:32:03.681955,303.781195,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-29 10:53:08.232747,351.712976,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
…,…,…,…,…,…,…
2024-08-23 08:02:25.193385,236.990536,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-23 01:37:31.451151,229.454328,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-28 22:23:33.615018,232.012233,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""


In [5]:
ts_to_sec = filtered_lf.with_columns(
    pl.col("ts").dt.strftime("%Y-%m-%d %H:%M:%S").str.strptime(pl.Datetime).alias("ts")
)

ts_to_sec.collect()

ts,value,source_key,metric_nice_name,uom,tank_type,tank_metric
datetime[μs],f64,str,str,str,str,str
2024-08-22 20:10:28,291.456951,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-26 18:34:13,298.883254,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-28 06:15:32,262.238814,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-26 09:32:03,303.781195,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-29 10:53:08,351.712976,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
…,…,…,…,…,…,…
2024-08-23 08:02:25,236.990536,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-23 01:37:31,229.454328,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-28 22:23:33,232.012233,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""


In [7]:
ts_to_sec = ts_to_sec.sort("ts")
ts_to_sec.collect()

ts,value,source_key,metric_nice_name,uom,tank_type,tank_metric
datetime[μs],f64,str,str,str,str,str
2024-08-22 17:57:10,77.524284,"""9883901""","""OilTank1Level""","""in""","""Oil""","""Level"""
2024-08-22 20:10:28,291.456951,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""
2024-08-22 21:19:35,224.297598,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-22 21:53:56,225.342223,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in""","""Oil""","""InchesUntilAlarm"""
2024-08-22 21:58:48,78.495096,"""9883901""","""OilTank1Level""","""in""","""Oil""","""Level"""
…,…,…,…,…,…,…
2024-08-29 07:42:38,272.901409,"""98839FAC""","""ESD-WaterTankInchesUntilAlarm""","""in""","""Water""","""InchesUntilAlarm"""
2024-08-29 09:20:45,277.054501,"""98839FAC""","""ESD-WaterTankInchesUntilAlarm""","""in""","""Water""","""InchesUntilAlarm"""
2024-08-29 10:53:08,351.712976,"""9883901""","""OilTank1Volume""","""bbl""","""Oil""","""Volume"""


In [8]:
aggregated_lf = ts_to_sec.group_by(["tank_metric", "tank_type","uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf = aggregated_lf.sort("tank_metric")

aggregated_lf.collect()

tank_metric,tank_type,uom,timestamps,values
str,str,str,list[datetime[μs]],list[f64]
"""InchesUntilAlarm""","""Oil""","""in""","[2024-08-22 21:19:35, 2024-08-22 21:53:56, … 2024-08-29 13:28:36]","[224.297598, 225.342223, … 276.817234]"
"""InchesUntilAlarm""","""Water""","""in""","[2024-08-23 06:29:52, 2024-08-23 17:26:52, … 2024-08-29 16:45:36]","[233.25937, 270.195369, … 266.361128]"
"""Level""","""Oil""","""in""","[2024-08-22 17:57:10, 2024-08-22 21:58:48, … 2024-08-29 07:29:23]","[77.524284, 78.495096, … 83.714711]"
"""Volume""","""Oil""","""bbl""","[2024-08-22 20:10:28, 2024-08-23 03:33:34, … 2024-08-29 10:53:08]","[291.456951, 300.210505, … 351.712976]"
